In [1]:
import glob
import pickle
import numpy as np
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

def train_network():
    """ Train a Neural Network to generate music """
    network_input, network_output = prepare_sequences(get_notes())
    validate_input, validate_output = prepare_sequences(get_notes("test"))

    model = create_network(network_input)

    train(model, network_input, network_output, validate_input, validate_output)
    
    
def merge_notes(notes_dict, max_offset):
            
    ret = np.array([])
    for i in np.arange(0, max_offset, 0.5):
        pitches = np.zeros(88)
        if i in notes_dict:            
            for element in notes_dict[i]:            
                if isinstance(element, note.Note):
                    pitches[element.pitch.midi-21] = 1
                else:
                    for p in element.pitches:
                        pitches[p.midi-21] = 1    
        ret = np.append(ret, pitches)
    return ret
    

def get_notes(folder="midi_songs"):
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = np.array([])

    for file in glob.glob("{}/*.mid".format(folder)):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes
        notes_dict = {}
        max_offset = 0
        for element in notes_to_parse:        
            if isinstance(element, note.Note) or isinstance(element, chord.Chord):  
                if element.offset not in notes_dict:
                    notes_dict[element.offset] = []
                notes_dict[element.offset].append(element)
                max_offset = element.offset    
        ret = merge_notes(notes_dict, max_offset)
        #print(len(ret))
        notes = np.append(notes, ret)
        #print(len(notes))
    row = notes.size / 88
    notes = notes.reshape(int(row), 88)
    #with open('data/notes', 'wb') as filepath:
        #pickle.dump(notes, filepath)
    
    return notes

def prepare_sequences(notes):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append(sequence_in)
        network_output.append(sequence_out)

    n_patterns = len(network_input)
    #print(n_patterns)
    # reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 88))
    network_output = np.reshape(network_output, (n_patterns, 88))
    return (network_input, network_output)

def create_network(network_input):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(88))
    #model.add(Activation('sigmoid'))
    #model.compile(loss='binary_crossentropy', optimizer='rmsprop')
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model

def train(model, network_input, network_output, validate_input,validate_output):
    """ train the neural network """
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]
    #print(network_input.shape)
    #print(network_output.shape)
    model.fit(network_input, network_output, 
              validation_data=(validate_input,validate_output), 
              epochs=200, batch_size=64, 
              callbacks=callbacks_list)


Using TensorFlow backend.


In [2]:
#n = get_notes()

In [3]:
train_network()

Parsing midi_songs/Ff7-Cinco.mid
Parsing midi_songs/waltz_de_choco.mid
Parsing midi_songs/Ff4-BattleLust.mid
Parsing midi_songs/dontbeafraid.mid
Parsing midi_songs/electric_de_chocobo.mid
Parsing midi_songs/costadsol.mid
Parsing midi_songs/Ff7-Jenova_Absolute.mid
Parsing midi_songs/tifap.mid
Parsing midi_songs/Still_Alive-1.mid
Parsing midi_songs/bcm.mid
Parsing midi_songs/ff7-mainmidi.mid
Parsing midi_songs/ultros.mid
Parsing midi_songs/0fithos.mid
Parsing midi_songs/mining.mid
Parsing midi_songs/Ff7-One_Winged.mid
Parsing midi_songs/FFIX_Piano.mid
Parsing midi_songs/balamb.mid
Parsing midi_songs/caitsith.mid
Parsing midi_songs/path_of_repentance.mid
Parsing midi_songs/ff1battp.mid
Parsing midi_songs/Finalfantasy6fanfarecomplete.mid
Parsing midi_songs/Oppressed.mid
Parsing midi_songs/relmstheme-piano.mid
Parsing midi_songs/ff4-fight1.mid
Parsing midi_songs/Finalfantasy5gilgameshp.mid
Parsing midi_songs/FF4.mid
Parsing midi_songs/Final_Fantasy_Matouyas_Cave_Piano.mid
Parsing midi_songs

Epoch 35/200
46310/46310 [==============================] - 168s 4ms/step - loss: 1.7355 - acc: 0.4851 - val_loss: 3.3218 - val_acc: 0.0125
Epoch 36/200
46310/46310 [==============================] - 168s 4ms/step - loss: 1.7279 - acc: 0.4866 - val_loss: 3.2753 - val_acc: 0.0150
Epoch 37/200
46310/46310 [==============================] - 168s 4ms/step - loss: 1.7249 - acc: 0.4864 - val_loss: 3.4214 - val_acc: 0.0125
Epoch 38/200
46310/46310 [==============================] - 168s 4ms/step - loss: 1.7153 - acc: 0.4855 - val_loss: 3.6023 - val_acc: 0.0250
Epoch 39/200
46310/46310 [==============================] - 168s 4ms/step - loss: 1.7088 - acc: 0.4861 - val_loss: 3.2016 - val_acc: 0.0100
Epoch 40/200
46310/46310 [==============================] - 168s 4ms/step - loss: 1.7085 - acc: 0.4870 - val_loss: 3.2329 - val_acc: 0.0200
Epoch 41/200
46310/46310 [==============================] - 168s 4ms/step - loss: 1.6970 - acc: 0.4866 - val_loss: 3.2531 - val_acc: 0.0175
Epoch 42/200
46310/4

46310/46310 [==============================] - 169s 4ms/step - loss: 1.5511 - acc: 0.4845 - val_loss: 2.9776 - val_acc: 0.0000e+00
Epoch 152/200
46310/46310 [==============================] - 168s 4ms/step - loss: 1.5030 - acc: 0.4872 - val_loss: 3.0502 - val_acc: 0.0025
Epoch 153/200
46310/46310 [==============================] - 168s 4ms/step - loss: 1.4999 - acc: 0.4905 - val_loss: 3.0012 - val_acc: 0.0075
Epoch 154/200
46310/46310 [==============================] - 168s 4ms/step - loss: 1.4988 - acc: 0.4910 - val_loss: 3.0308 - val_acc: 0.0025
Epoch 155/200
46310/46310 [==============================] - 168s 4ms/step - loss: 1.4967 - acc: 0.4888 - val_loss: 3.0775 - val_acc: 0.0100
Epoch 156/200
46310/46310 [==============================] - 168s 4ms/step - loss: 1.5003 - acc: 0.4897 - val_loss: 3.0655 - val_acc: 0.0150
Epoch 157/200
46310/46310 [==============================] - 168s 4ms/step - loss: 1.4942 - acc: 0.4902 - val_loss: 3.0661 - val_acc: 0.0125
Epoch 158/200
46310/463

In [6]:
""" This module generates notes for a midi file using the
    trained neural network """
import pickle
import numpy as np
from music21 import instrument, note, stream, chord, pitch
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation

sequence_length = 100
def get_notes(folder="test"):
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = np.array([])

    for file in glob.glob("{}/*.mid".format(folder)):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes
        notes_dict = {}
        max_offset = 0
        for element in notes_to_parse:        
            if isinstance(element, note.Note) or isinstance(element, chord.Chord):  
                if element.offset not in notes_dict:
                    notes_dict[element.offset] = []
                notes_dict[element.offset].append(element)
                max_offset = element.offset    
        ret = merge_notes(notes_dict, max_offset)
        #print(len(ret))
        notes = np.append(notes, ret)
        #print(len(notes))
    row = notes.size / 88
    notes = notes.reshape(int(row), 88)
    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)
    
    return notes

def generate():
    """ Generate a piano midi file """
    #load the notes used to train the model
    #with open('data/notes', 'rb') as filepath:
    #    notes = pickle.load(filepath)

    network_input = prepare_sequences(get_notes())
    model = create_network()
    prediction_output = generate_notes(model, network_input)
    create_midi(prediction_output)

def prepare_sequences(notes):
    """ Prepare the sequences used by the Neural Network """
    
    network_input = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        network_input.append(sequence_in)

    return network_input

def create_network():
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(sequence_length, 88),
        return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(88))
    #model.add(Activation('sigmoid'))
    #model.compile(loss='binary_crossentropy', optimizer='rmsprop')
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    # Load the weights to each node
    model.load_weights('weights-improvement-195-1.4606-bigger.hdf5')
    return model

def generate_notes(model, network_input):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = np.random.randint(0, len(network_input)-1)
    pattern = network_input[start]
    prediction_output = []
    threshold = 0.5
    # generate 500 notes
    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, sequence_length, 88))
        prediction = model.predict(prediction_input, verbose=0)[0]
        index = np.argmax(prediction)
        prediction = np.zeros(88)
        prediction[index] = 1
        pattern = np.append(pattern, prediction)[88:]    
        #print(prediction)
        prediction_output.append(prediction)
    return prediction_output

import time
current_time = lambda: int(round(time.time()))

def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []
    #print(prediction_output)
    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        indices = np.where(pattern > 0)[0]
        if len(indices) > 0:
            notes = []
            for index in indices:
                n = note.Note()
                p = pitch.Pitch()
                p.midi = index + 21
                n.pitch = p
                n.storedInstrument = instrument.Piano()
                notes.append(n)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
       
        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='40_sentence_05_test_output_{}.mid'.format(current_time()))
    

In [7]:
for i in range(20):
    generate()

Parsing test/chpn_op25_e4.mid
Parsing test/chpn_op25_e1.mid
Parsing test/chpn_op25_e3.mid
Parsing test/chpn_op25_e2.mid
Parsing test/chpn_op25_e4.mid
Parsing test/chpn_op25_e1.mid
Parsing test/chpn_op25_e3.mid
Parsing test/chpn_op25_e2.mid
Parsing test/chpn_op25_e4.mid
Parsing test/chpn_op25_e1.mid
Parsing test/chpn_op25_e3.mid
Parsing test/chpn_op25_e2.mid
Parsing test/chpn_op25_e4.mid
Parsing test/chpn_op25_e1.mid
Parsing test/chpn_op25_e3.mid
Parsing test/chpn_op25_e2.mid
Parsing test/chpn_op25_e4.mid
Parsing test/chpn_op25_e1.mid
Parsing test/chpn_op25_e3.mid
Parsing test/chpn_op25_e2.mid
Parsing test/chpn_op25_e4.mid
Parsing test/chpn_op25_e1.mid
Parsing test/chpn_op25_e3.mid
Parsing test/chpn_op25_e2.mid
Parsing test/chpn_op25_e4.mid
Parsing test/chpn_op25_e1.mid
Parsing test/chpn_op25_e3.mid
Parsing test/chpn_op25_e2.mid
Parsing test/chpn_op25_e4.mid
Parsing test/chpn_op25_e1.mid
Parsing test/chpn_op25_e3.mid
Parsing test/chpn_op25_e2.mid
Parsing test/chpn_op25_e4.mid
Parsing te